## Проект для «Викишоп»

### Задача

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, то есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

In [1]:
#Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import cross_val_score
from pymystem3 import Mystem
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#Загружаем файл с данными
data = pd.read_csv('/datasets/toxic_comments.csv')
display(data.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
#Лемматизируем текст (время исполнения ячейки 2,5 минуты)
m = Mystem()
def token_text (row):
    category = m.lemmatize(row)
    row = ''.join(category)
    return row.rstrip('\n')
#data['tokenized']= data['text'].apply(token_text)

In [4]:
#Очистим данные от лишних символов
def clear (row):
    w = re.sub(r'[^a-zA-Z]',' ',row)
    n = " ".join(w.split())
    return n
data['clean_text']= data['text'].apply(clear)
display(data['clean_text'].head())

0    Explanation Why the edits made under my userna...
1    D aww He matches this background colour I m se...
2    Hey man I m really not trying to edit war It s...
3    More I can t make any real suggestions on impr...
4    You sir are my hero Any chance you remember wh...
Name: clean_text, dtype: object

In [5]:
#Создадим счетчик слов tfid idf, указав стоп слова, для очистки текста
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords, lowercase=True, min_df=0.0001)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#Обозначим признаки и таргет
features = data['clean_text']
target = data['toxic']

In [7]:
#Разделим данные на обучающую и тестовую выборки в соотношении 95/5
features_train, features_test, target_train,target_test = train_test_split(features, target, test_size = 0.05, random_state=12345)

In [8]:
#Переведем подготовленный текст в формат unicode
features_train = features_train.values.astype('U')
features_test = features_test.values.astype('U')

In [9]:
#Обучим векторизатор на тренировочной части данных, переведем в вектора обе
tf_idf_features_train = count_tf_idf.fit_transform(features_train)
tf_idf_features_test = count_tf_idf.transform(features_test)

## Обучение

In [10]:
#Логистическая регрессия
model_lr = LogisticRegression(random_state = 12345)
print('f1 логистической регрессии, кросс-валидация на обучающей выборке',(
    cross_val_score(model_lr, tf_idf_features_train, target_train, cv=3, scoring = 'f1')).mean())
model_lr.fit(tf_idf_features_train,target_train)
predict_lr = model_lr.predict(tf_idf_features_test)
print()
print('f1 логистической регрессии на тестовой выборке', f1_score(predict_lr,target_test))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


f1 логистической регрессии, кросс-валидация на обучающей выборке 0.7322415411521103

f1 логистической регрессии на тестовой выборке 0.7514792899408285


In [11]:
#Модель дерева
model_dt = DecisionTreeClassifier(random_state = 12345, max_depth = 15)
print('f1 модели дерева',(cross_val_score(model_dt,tf_idf_features_train, target_train, cv=3, scoring = 'f1')).mean())

f1 модели дерева 0.6155757180741442


In [12]:
#Лес деревьев
model_rf = RandomForestClassifier(random_state = 12345, n_estimators = 2, max_depth = 10)
print('f1 лес деревьев',(cross_val_score(model_rf,tf_idf_features_train,target_train, cv=3, scoring = 'f1')).mean())

f1 лес деревьев 0.2479645913572804


## Выводы

Модель линейной регрессии, после предварительной подготовки признаков (токенизация, лемматизация и очищение от не несущих смысловую нагрузку слов, показывает лучший результат метрики f1, удовлетворяющий условиям задачи. Для достижения результата соотношение трейновой к тестовой увеличил в соотношении 95% к 5% - ограничений на этот счёт в условиях не было